# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [217]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [218]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [219]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [220]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [221]:
players_180_190 = player_data['height'].between(180,190).sum()

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [222]:
players_1980 = (pd.to_datetime(player_data['birthday']).dt.year == 1980).sum()

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [223]:
highest_players = player_data.sort_values(['weight', 'player_name'], ascending=(False, True))['player_name'].head(10).to_list()

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [224]:
player_data1 = player_data
player_data1['year'] = pd.to_datetime(player_data['birthday']).dt.year
player_year_series = player_data1[player_data1['year'].between(1980,1990)].groupby('year').size()
years_born_players = list(zip(player_year_series.index, player_year_series))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [225]:
adriano_mean, adriano_std = player_data[player_data['player_name'].str.startswith('Adriano')]['height'].agg(['mean', 'std']).values

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [226]:
player_data2 = player_data
player_data2['day_of_birth'] = pd.to_datetime(player_data['birthday']).dt.day_name()
dow_with_min_players_born = player_data2.groupby('day_of_birth').size().idxmin()

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [227]:
match_data = pd.read_sql("SELECT * FROM Match;", db)
league_data = pd.read_sql("SELECT * FROM League;", db)
dfg = match_data.groupby('league_id', as_index=False, sort=False).size()
max_matches = dfg['size'].max()
mdfg = dfg[dfg['size'] == max_matches]
mdfg1 = mdfg.rename(columns={'league_id':'id'})

league_most_matches = pd.merge(league_data, mdfg1).sort_values('name').iloc[0]['name']

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [228]:
home_player = 'home_player_'
away_player = 'away_player_'
home_players = [item+str(index) for index, item in enumerate(list([home_player]*11), start=1)]
away_players = [item+str(index) for index,
                item in enumerate(list([away_player]*11), start=1)]
players = home_players + away_players
# match_data.dtypes
ps = [match_data[item].dropna().astype('int64') for item in players]
most_freq = pd.concat(ps).mode()
max_matches_player = player_data[player_data['player_api_id'] ==
int(most_freq)].iloc[0]['player_name']


In [229]:
player_attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [250]:
pure_player_attributes_data = player_attributes_data.drop(columns=['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'])
cor_df = pure_player_attributes_data.corr().abs()
cor_df = cor_df.unstack().reset_index()
cor_df.columns = ['character_1', 'character_2', 'correlation']
mask_dups = (cor_df[['character_1', 'character_2']].apply(frozenset, axis=1).duplicated()) | (cor_df['character_1']==cor_df['character_2']) 
cor_df = cor_df[~mask_dups]
sorted_cor_df = cor_df.sort_values(by=['correlation'], ascending=False)
cor_df_top_5 = sorted_cor_df.reset_index()[['character_1', 'character_2']].iloc[:5]
cor_df_top_5
# top_correlated_features = list(cor_df_top_5.itertuples(index=False, name=None))

,character_1,character_2
0,gk_positioning,gk_reflexes
1,gk_handling,gk_reflexes
2,gk_handling,gk_positioning
3,standing_tackle,sliding_tackle
4,marking,standing_tackle


**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [298]:
characters_list = ['gk_positioning', 'gk_reflexes', 'gk_handling', 'standing_tackle', 'sliding_tackle', 'marking' ]
player_attributes_data_cast = player_attributes_data[['player_api_id', 'date'] + characters_list]
player_attributes_data_cast['date'] = pd.to_datetime(player_attributes_data_cast['date'])

neymar_id = player_data.loc[(player_data['player_name'].str.startswith('Neymar'))].reset_index().at[0,'player_api_id']

fresh_payer_id_data = player_attributes_data_cast.groupby(by='player_api_id')['date'].agg(['max']).reset_index()
neymar_fresh_data = fresh_payer_id_data[fresh_payer_id_data['player_api_id']==neymar_id].iat[0, 1]
neymar_characteristics = player_attributes_data_cast[(player_attributes_data_cast['player_api_id'] == neymar_id) & (player_attributes_data_cast['date'] == neymar_fresh_data)]
fresh_payer_id_data = fresh_payer_id_data.drop(fresh_payer_id_data[fresh_payer_id_data['player_api_id'] == neymar_id].index)
for  player_id, fresh_date in fresh_payer_id_data.itertuples(index=False, name=None):
    player_attributes_data_cast.loc[(player_attributes_data_cast['player_api_id'] == player_id) & (player_attributes_data_cast['date'] == fresh_date)]
    
# player_attributes_data_cast[player_attributes_data_cast['player_api_id'] == neymar_id]
# neymar_similarities = 0

C:\Users\slobbery.GBAS\AppData\Local\Temp\ipykernel_216\3017346041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_attributes_data_cast['date'] = pd.to_datetime(player_attributes_data_cast['date'])


KeyboardInterrupt: 

In [232]:
team_data = pd.read_sql("SELECT * FROM Team;", db)

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [233]:
borussia_id = team_data.loc[(team_data['team_long_name'] == 'Borussia Dortmund')].reset_index().at[0,'team_api_id']
g1_bundesliga_id = league_data.loc[(league_data['name'] == 'Germany 1. Bundesliga')].reset_index().at[0,'id']
season = '2008/2009'
borussia_bundesliga_2008_2009_matches = len(match_data[(match_data['league_id']==g1_bundesliga_id) & (match_data['season']==season) & (match_data['home_team_api_id']==borussia_id)])


**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [234]:
g1_bundesliga_season = match_data[(match_data['league_id']==g1_bundesliga_id) & (match_data['season']==season)]
home_teams = g1_bundesliga_season['home_team_api_id'].value_counts()
away_teams = g1_bundesliga_season['away_team_api_id'].value_counts()
team_most_matches_bundesliga_2008_2009 = home_teams.add(away_teams, fill_value=0).sort_values().iloc[0]

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [235]:
season_2015 = '2015/2016'
arsenal_id = team_data.loc[(team_data['team_long_name'] == 'Arsenal')].reset_index(drop=True).at[0,'team_api_id']
arsenal_won_matches = match_data[(match_data['season']==season_2015) & (((match_data['home_team_api_id']==arsenal_id) & (match_data['home_team_goal'] > match_data['away_team_goal'])) | ((match_data['away_team_api_id']==arsenal_id) & (match_data['away_team_goal'] > match_data['home_team_goal'])))]
arsenal_won_matches_2015_2016 = len(arsenal_won_matches)

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [243]:
matches_2015 = match_data[match_data['season']==season_2015]
matches_2015_home = matches_2015[['home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']]
matches_2015_away = matches_2015_home.copy()
matches_2015_home1 = matches_2015_home.rename(columns={'home_team_api_id':'team_api_id', 'away_team_api_id':'alient_team_api_id', 'home_team_goal':'team_goal','away_team_goal':'alient_team_goal' })
matches_2015_away1 = matches_2015_away.rename(columns={'away_team_api_id':'team_api_id', 'home_team_api_id':'alient_team_api_id', 'away_team_goal':'team_goal','home_team_goal':'alient_team_goal' })
whole_matches = pd.concat([matches_2015_home1, matches_2015_away1])
whole_matches['winner'] = whole_matches.apply(lambda x: 1 if x['team_goal'] > x['alient_team_goal'] else 0, axis=1)
winner_df = whole_matches.groupby('team_api_id')['winner'].agg(['sum','count'])
winner_df['rate'] = winner_df.apply(lambda x: x['sum']/x['count'], axis=1)
team_highest_winrate_2015_2016 = winner_df.sort_values(by='rate', ascending=False).merge(team_data, on='team_api_id').at[0, 'team_long_name']

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [246]:
season_2010 = '2010/2011'
en_league_id = league_data.loc[(league_data['name'] == 'England Premier League')].reset_index().at[0,'id']
en_league_id
# highest_gap_england_2010_2011 = 0

1729

### Warning! Do not change anything in the area below

In [238]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")